In [1]:
import numpy as np
from BPnumba.Individual import Ind
from BPmodule.BPGen import GetInstance
from BPnumba.AGH import GASearch
import matplotlib.pyplot as plt
from numba import types, njit,int64,jit,cuda,prange,vectorize
from numba.typed import List as NumbaList
from BPnumba.Poblation import CreateRotHPob,EvalPob
from BPnumba.BoxN import create_ItemBin,ItemBin,InstaceBoxes
from BPnumba.GeneticOperators import CrossOX,MutateInversion,RepairRan
from BPnumba.BPPH import create_Bin,DBLF
from BPmodule.DataLecture import ExpotarJSON,PossiblePositions
import math

In [2]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)
createText = lambda x,y : np.savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g'
)

In [3]:
@njit#(parallel=True)
def BenchAG(i,boxes,bin,lst,lst2,lst3,maxItr,pop,rot):
    #fi = NumbaList(np.zeros(maxItr,dtype=np.float64))
    UBOXES = InstaceBoxes(boxes)
    pob = CreateRotHPob(pop =pop, ITEMS = UBOXES, rotType=rot)
    EvalPob(pob,UBOXES,bin,rot)
    
    ind:Ind=GASearch(maxItr=maxItr,ps=0.85,pc=0.75,pmr=0.05,mut=0,pob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rotType=rot)
    ind2:Ind=GASearch(maxItr=maxItr,ps=0.85,pc=0.75,pmr=0.05,mut=1,pob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rotType=rot)
    ind3:Ind=GASearch(maxItr=maxItr,ps=0.85,pc=0.75,pmr=0.05,mut=2,pob=pob.copy(),datos=UBOXES.copy(),contenedor=bin,rotType=rot)
    
    lst[i]=ind.fi
    lst2[i] = ind2.fi
    lst3[i] = ind3.fi
@njit(parallel=True)
def BAG(rango,rango2,contenedor,data:list[list[list[int]]],agL,agL2,agL3,maxItr,numpop,rotation):
    for p in prange(rango,rango2):
        BenchAG(
            i=p,
            boxes=data[p], 
            bin=contenedor,
            maxItr= maxItr,
            lst= agL,
            lst2=agL2,
            lst3=agL3,
            pop=numpop,
            rot=rotation)

In [4]:
def Bench(nm,r,subthread=True,n=4):
    BR1 = GetInstance(nm)
    agL = zerovec(1, 100)
    agL2 = zerovec(1, 100)
    agL3 = zerovec(1, 100)
    if subthread:
        for th in np.arange(0,r,4):
            BAG(rango=th,rango2=th+n,contenedor=BR1[0],data=BR1[1],agL=agL,agL2=agL2,agL3=agL3,maxItr=1000,numpop=20,rotation=6)
            createText(nm+"/AG/"+"1AG"+str(0),agL)
            createText(nm+"/AG/"+"1AG"+str(1),agL2)
            createText(nm+"/AG/"+"1AG"+str(2),agL3)
    else:
        BAG(rango=0,rango2=r,contenedor=BR1[0],data=BR1[1],agL=agL,agL2=agL2,agL3=agL3,maxItr=1000,numpop=20,rotation=6)
        createText(nm+"/AG/"+"1AG"+str(0),agL)
        createText(nm+"/AG/"+"1AG"+str(1),agL2)
        createText(nm+"/AG/"+"1AG"+str(2),agL3)

In [5]:
BR1 = GetInstance("BR1")

a:\Documents\python\BPP3D\BPmodule\BPGen.py:405: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bx= np.array(totalboxes)


In [6]:
UBOXES = InstaceBoxes(BR1[1][1])
rot=2
Pob = CreateRotHPob(pop =10, ITEMS = UBOXES.copy(), rotType=rot)

In [7]:
len(UBOXES)

212

In [29]:
n= len(BR1[1][0])
pop= 40
maxItr=100
rot=2
pr_selec=0.85
pr_cruza = 0.75
pr_mutR=0.05
mutType=0
n,pop

(310, 30)

In [31]:
EvalPob(Pob,UBOXES,BR1[0],rot)

In [32]:
Pob[0].genome,Pob[0].fi

(ListType[int64]([136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 161, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 148, 135, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 134, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 122, 160, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 187, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 201, 162, 188, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 186, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 173, 175, 107, 109, 108, 106, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 77, 64, 62, 61, 60, 59, 58, 57, 56, 55, 54, 52, 51, 50, 63, 53, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 78, 79, 80, 81, 82, 83, 84, 92, 91, 90, 89, 85, 88, 87, 86, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 12, 25, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 13, 27, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 1, 26, ...]),
 0.8376313825166286)

In [33]:
fi = NumbaList(np.zeros(maxItr,dtype=np.float64))
ind = GASearch(maxItr=maxItr,ps=pr_selec,pc=pr_cruza,pmr=pr_mutR,mut=mutType,pob=Pob.copy(),datos=UBOXES.copy(),contenedor=BR1[0],rotType=rot )

In [34]:
ind.fi

0.8659488222184262

In [35]:
ind.genome,ind.genome_r

(ListType[int64]([155, 156, 157, 158, 159, 148, 135, 110, 111, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 122, 160, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 187, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 201, 162, 188, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 186, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 173, 175, 25, 141, 137, 9, 3, 8, 145, 11, 143, 146, 138, 139, 2, 142, 7, 5, 140, 4, 13, 112, 113, 114, 115, 134, 120, 119, 116, 118, 117, 94, 95, 96, 97, 98, 99, 101, 93, 102, 103, 104, 105, 53, 100, 63, 79, 80, 81, 82, 83, 84, 92, 91, 90, 89, 85, 88, 87, 86, 78, 108, 107, 109, 106, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 50, 51, 52, 54, 55, 56, 57, 65, 77, 64, 62, 58, 61, 60, 59, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 37, 27, 36, 35, 34, 33, 32, 31, 30, 29, 28, 1, 38, 26, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 12, 136, 6, 10, 161, 144, 147, 149, 150, 151, 152, 153, 154, ...]),
 ListType[int64]([1, 0,

In [36]:
bin = create_Bin(NumbaList(BR1[0]))

In [37]:
DBLF(
        bin = bin, 
        itemsToPack= ind.genome,
        itemsRor= ind.genome_r,
        BoxesData = UBOXES,
        wayRotation=rot
        ) #Implementa heuristica

In [38]:
path=r"A:/Documents/Unity/BPP/Assets/bpp.json"
path2=r"A:/Documents/Unity/BPP/Assets/points.json"
ExpotarJSON(path=path,rot=rot,Cont=bin,Data=UBOXES)
PossiblePositions(path=path2,positions=bin.extrapts)

Done!
